# Module 1 - Challenge
### Creating a Customer Service Chatbot

This notebook implements a prompt-engineered customer service chatbot for TechStore Plus.

### 1. Setup & API Configuration

In [ ]:
# Install necessary packages

!pip install openai
!pip install python-dotenv

In [2]:
# Import necessary libraries

import os
import openai
from dotenv import load_dotenv
import json
import datetime
from uuid import uuid4 # Generate unique IDs for each conversation


# Load environment variables from .env file

load_dotenv()

openai.api_key  = os.getenv('OPENAI_API_KEY')

client = openai.OpenAI()

### 2. System Prompt

In [3]:
# Define the system prompt for the chatbot
# This prompt sets the context and behavior of the chatbot
# It should be clear, concise, and provide all necessary information about the company and its services 

system_prompt = """
You are a helpful and friendly assistant for TechStore Plus, an e-commerce for technology producs, which has an online store and physical showrrom based in New York, USA founded in 2018.

Company Info:
- Business hours: Mon-Fri 9am-6pm, Sat 10am-2pm
- Product Categories: Laptops, desktops, smartphones, accessories, gaming, smart home devices
- Services: Sales, tech support, installation, financing.
- Shipping: Free for orders over $500
- Returns: 30 days for exchange, 7 days for refunds. It has a warranty available for 12 months on all the producs and also a extended warranty option for 1 additional year
- Contact: support@techstoreplus.com, 1-800-TECH-PLUS, website chat (24/7), text +1 555-123-4567

Always be clear, concise, and offer support options if needed.
"""

### 3. Functions

#### 3.1 Chat Function

In [49]:
# Function to get a completion from the model

def get_completion(prompt, model="gpt-4.1"):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
                ]
    response = client.chat.completions.create(
        model=model,
        messages=messages, 
        temperature=0,  #degreee of randomness in the response
    )
    return response.choices[0].message.content

#### 3.2 Conversation Management Function

In [50]:
# Function to continue the conversation with the user

def continue_conversation(conversation, user_message):
    conversation.append({"role": "user", "content": user_message})
    reply = get_completion(conversation)
    conversation.append({"role": "assistant", "content": reply})
    return conversation, reply

In [51]:
def continue_conversation(conversation, user_message):
    """
    Continue the conversation with a new user message.
    Args:
        conversation (list): Conversation history.
        user_message (str): The user's message.
    Returns:
        tuple: Updated conversation and assistant's reply.
    """
    conversation.append({"role": "user", "content": user_message})
    # Build messages for context
    messages = [{"role": "system", "content": system_prompt}] + conversation
    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=messages,
        temperature=0,
        max_tokens=300
    )
    reply = response.choices[0].message.content
    conversation.append({"role": "assistant", "content": reply})
    return conversation, reply

#### 3.3 Query Analysis Function

In [ ]:
# Function to analyze the user message

def analyze_query_with_gpt(user_message):
    prompt = f"""
You are an intelligent assistant that analyzes customer service messages.

Analyze the following message and return a JSON with:
- sentiment: positive / neutral / negative
- customer_id: autogenenerated
- conversation_summary: a brief and concise summary of the conversation
- query_category: technical / billing / returns / general_information / sales / warranty / installation
- customer_sentiment: frustration, urgency, satisfaction
- urgency_level: low / medium / high
- mentioned_products: list any tech products
- extracted_information: extract [order_number, purchase_date, amount] (if applicable)
- resolution_status: resolved / pending / escalated
- actions_taken: list any actions taken by the assistant
- follow_up_needed: true / false

Answer within a sentence and just if the user request give an action to take, otherwise just analyze the message.

Message: '''{user_message}'''

Respond ONLY in valid JSON format.
"""
    response = get_completion(prompt)
    return json.loads(response)

#### 3.4 Conversation Summary Functions 

In [ ]:
# Function to summarize the conversation

def get_conversation_summary(chat_history):
    """
    Use GPT to generate a brief summary of the conversation between user and bot.
    """
    formatted_history = ""
    for msg in chat_history:
        role = msg["role"].capitalize()
        formatted_history += f"{role}: {msg['content']}\n"
    prompt = (
        "Summarize the following conversation between a customer and a support assistant in 2-3 sentences, "
        "highlighting the main issue, actions taken, and any next steps:\n\n"
        f"{formatted_history}"
    )
    return get_completion(prompt)


# Function to summarize the conversation with detailed structured information

def summarize_conversation(
        chat_history, 
        summary, 
        category, 
        sentiment, 
        urgency, 
        mentioned_products, 
        extracted_info, 
        resolution_status, 
        actions_taken, 
        follow_up_required):
    return {
        "timestamp": datetime.datetime.now().isoformat(),
        "customer_id": str(uuid4()),
        "conversation_summary": summary,
        "query_category": category,
        "customer_sentiment": sentiment,
        "urgency_level": urgency,
        "mentioned_products": mentioned_products,
        "extracted_information": extracted_info,
        "resolution_status": resolution_status,
        "actions_taken": actions_taken,
        "follow_up_required": follow_up_required
    }

In [54]:
# Function to summarize the conversation with analysis
# This function combines the conversation history with the analysis of the user's message to create a comprehensive summary

def summarize_with_analysis(conversation, user_message, chat_response):
    analysis = analyze_query_with_gpt(user_message)

    summary = summarize_conversation(
        chat_history=conversation,
        summary=chat_response[:100],
        category=analysis["category"],
        sentiment=analysis["sentiment"],
        urgency=analysis["urgency"],
        mentioned_products=analysis.get("mentioned_products", []),
        extracted_info=analysis.get("extracted_information", {}),
        resolution_status=analysis['resolution_status'],
        actions_taken=["auto-response generated"],
        follow_up_required=analysis["urgency"] == "high"
    )
    return summary

#### 3.5 Save-to-File (JSON format) Function

In [55]:
# Function to save the summary to a file
# This function saves the conversation summary to a JSON file in a specified directory.

def save_summary_to_file(summary_data, directory="conversations"):
    os.makedirs(directory, exist_ok=True)
    filename = f"{directory}/conv_{summary_data['timestamp'].replace(':', '-')}.json"
    with open(filename, 'w') as f:
        json.dump(summary_data, f, indent=4)
    print(f"✅ Saved: {filename}")

#### 4. Test Queries

In [56]:
test_queries = [
    {
        "label": "Neutral-Informative",
        "user_message": "Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago",
        "expected": {
            "query_category": "information",
            "sentiment": "neutral",
            "urgency_level": "low",
            "mentioned_products": ["iPhone 15"],
            "extracted_information": {
                "order_number": None,
                "purchase_date": None,
                "amount": None
            }
        }
    },
    {
        "label": "Urgent-Negative",
        "user_message": "This is an emergency! My order #TEC-2024-001 never arrived and I need that laptop for work tomorrow!",
        "expected": {
            "query_category": "billing",
            "sentiment": "negative",
            "urgency_level": "high",
            "mentioned_products": ["Laptop"],
            "extracted_information": {
                "order_number": "TEC-2024-001",
                "purchase_date": None,
                "amount": None
            }
        }
    },
    {
        "label": "Satisfied-Positive",
        "user_message": "Thank you so much for the excellent service with my previous purchase, I want to buy gaming headphones",
        "expected": {
            "query_category": "sales",
            "sentiment": "positive",
            "urgency_level": "low",
            "mentioned_products": ["Gaming Headphones"],
            "extracted_information": {}
        }
    },
    {
        "label": "Frustrated-Technical",
        "user_message": "I can't configure the router I bought last week, I've tried everything and it doesn't work",
        "expected": {
            "query_category": "technical",
            "sentiment": "negative",
            "urgency_level": "medium",
            "mentioned_products": ["Router"],
            "extracted_information": {}
        }
    },
    {
        "label": "Formal-Billing",
        "user_message": "Good morning, I need the receipt for my purchase from December 15th, order #TEC-2023-089",
        "expected": {
            "query_category": "billing",
            "sentiment": "neutral",
            "urgency_level": "low",
            "mentioned_products": [],
            "extracted_information": {
                "order_number": "TEC-2023-089",
                "purchase_date": "2023-12-15",
                "amount": None
            }
        }
    },
    {
        "label": "Warranty-Query",
        "user_message": "I bought a tablet 8 months ago and now it won't turn on, how do I use the warranty?",
        "expected": {
            "query_category": "technical",
            "sentiment": "neutral",
            "urgency_level": "medium",
            "mentioned_products": ["Tablet"],
            "extracted_information": {}
        }
    },
    {
        "label": "Product-Information",
        "user_message": "What laptop do you recommend for an engineering student? Maximum budget $800",
        "expected": {
            "query_category": "information",
            "sentiment": "neutral",
            "urgency_level": "low",
            "mentioned_products": ["Laptop"],
            "extracted_information": {
                "amount": 800
            }
        }
    },
    {
        "label": "Urgent-Return",
        "user_message": "I need to urgently return this phone I bought yesterday, it's not compatible with my plan",
        "expected": {
            "query_category": "return",
            "sentiment": "negative",
            "urgency_level": "high",
            "mentioned_products": ["Phone"],
            "extracted_information": {}
        }
    },
    {
        "label": "Installation-Support",
        "user_message": "Can you come install the home theater I bought? I live in downtown",
        "expected": {
            "query_category": "technical",
            "sentiment": "neutral",
            "urgency_level": "medium",
            "mentioned_products": ["Home Theater"],
            "extracted_information": {}
        }
    },
    {
        "label": "Financing",
        "user_message": "Do you have interest-free installments for the MacBook Pro? Do you accept Visa cards?",
        "expected": {
            "query_category": "information",
            "sentiment": "positive",
            "urgency_level": "low",
            "mentioned_products": ["MacBook Pro"],
            "extracted_information": {}
        }
    }
]

In [ ]:
# Display the test queries

for i in range(10):
    print(f"{i+1}. {test_queries[i]['user_message']}")

1. Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago
2. This is an emergency! My order #TEC-2024-001 never arrived and I need that laptop for work tomorrow!
3. Thank you so much for the excellent service with my previous purchase, I want to buy gaming headphones
4. I can't configure the router I bought last week, I've tried everything and it doesn't work
5. Good morning, I need the receipt for my purchase from December 15th, order #TEC-2023-089
6. I bought a tablet 8 months ago and now it won't turn on, how do I use the warranty?
7. What laptop do you recommend for an engineering student? Maximum budget $800
8. I need to urgently return this phone I bought yesterday, it's not compatible with my plan
9. Can you come install the home theater I bought? I live in downtown
10. Do you have interest-free installments for the MacBook Pro? Do you accept Visa cards?


### 5. Sample Response

In [58]:
first_query = test_queries[0]["user_message"]
print("User query:", first_query)
conversation_history = []
conversation_history, bot_reply = continue_conversation(conversation_history, first_query)
print("\n🤖 Bot:", bot_reply)

analysis = analyze_query_with_gpt(first_query)
print("\n\nAnalysis:\n", json.dumps(analysis, indent=2))

summary = summarize_conversation(
    chat_history=conversation_history,
    summary=get_conversation_summary(conversation_history),
    category=analysis.get("query_category"),
    sentiment=analysis.get("customer_sentiment"),
    urgency=analysis.get("urgency_level"),
    mentioned_products=analysis.get("mentioned_products", []),
    extracted_info=analysis.get("extracted_information", {}),
    resolution_status=analysis.get("resolution_status", "pending"),
    actions_taken=analysis.get("actions_taken", ["auto-response generated"]),
    follow_up_required=analysis.get("urgency_level", "") == "high"
)
print("\nSummary:\n", json.dumps(summary, indent=2))

User query: Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago

🤖 Bot: Hello! Yes, we have the new iPhone 15 in stock at TechStore Plus. 

Shipping to Chicago is **free** for orders over $500. Since the iPhone 15 is priced above this amount, you’ll receive free shipping on your order.

If you’d like to check specific models, colors, or storage options, or if you need help placing an order, please let me know!


Analysis:
 {
  "sentiment": "neutral",
  "customer_id": "CUST-20240610-001",
  "conversation_summary": "Customer inquired about the availability of the iPhone 15 and shipping costs to Chicago.",
  "query_category": "sales",
  "customer_sentiment": "neutral",
  "urgency_level": "low",
  "mentioned_products": [
    "iPhone 15"
  ],
  "extracted_information": {
    "order_number": null,
    "purchase_date": null,
    "amount": null
  },
  "resolution_status": "pending",
  "actions_taken": [],
  "follow_up_needed": true
}

Summary:


### 6. Export all Samples to the 'conversations' folder

In [59]:
conversation_history = []
for query in test_queries:
    print(f"Testing: {query['label']}")
    conversation_history, bot_reply = continue_conversation(conversation_history, query["user_message"])
    print("🤖 Bot:", bot_reply)
    analysis = analyze_query_with_gpt(query["user_message"])
    summary = summarize_conversation(
        chat_history=conversation_history,
        summary=get_conversation_summary(conversation_history),
        category=analysis.get("query_category"),
        sentiment=analysis.get("customer_sentiment"),
        urgency=analysis.get("urgency_level"),
        mentioned_products=analysis.get("mentioned_products", []),
        extracted_info=analysis.get("extracted_information", {}),
        resolution_status=analysis.get("resolution_status", "pending"),
        actions_taken=analysis.get("actions_taken", ["auto-response generated"]),
        follow_up_required=analysis.get("urgency_level", "") == "high"
    )
    save_summary_to_file(summary)
    print("✅ Test passed!\n")

Testing: Neutral-Informative
🤖 Bot: Hello! Yes, we have the new iPhone 15 in stock at TechStore Plus. Shipping to Chicago is **free** for orders over $500, which applies to the iPhone 15. If your order is under $500, standard shipping rates will apply.

Would you like more details on the available models, colors, or storage options? If you need help placing an order or have any other questions, please let me know!
✅ Saved: conversations/conv_2025-07-30T23-34-59.247804.json
✅ Test passed!

Testing: Urgent-Negative
🤖 Bot: I'm very sorry to hear about this urgent situation. Let me help you right away.

Here’s what you can do:
1. **Check your order status:** You can log in to your TechStore Plus account and view the latest tracking information.
2. **Contact us immediately:**  
   - **Call:** 1-800-TECH-PLUS (for fastest assistance during business hours)  
   - **Chat:** Use our 24/7 website chat for instant support  
   - **Text:** +1 555-123-4567

I recommend calling or using the website 